<h5 style="color:green"> All Models' features need to be 2D Arrays even if there is one feature </h3>
<ul>
<li> No documnetation
<li> Better OOP Design and Redundancy Omitting
<li> To be implemented :
    <ul>
        <li> Gaussian Naive Bayes
        <li> Logistic Regression
        <li> Support Vector Machine
        <li> Gradient Boosting
        <li> DBSCAN and HDBSCAN
        <li> K-Means
        <li> PCA - UMAP
        <li> Reinforcement Learning
        <li> AlphaZero
        <li> Factorization Methods
        <li> Convolutional Neural Networks
        <li> RNN + LSTM
        <li> Transformers
    </ul>
<li> Needs Better Implementations :
    <ul>
        <li> Faster BallTree / KDTree Algorithms for KNN
    </ul>
</ul>

In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier

import decision_tree
import random_forest
import linear_regression
import naive_bayes
import nearest_neighbors

from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pickle

import time
import matplotlib.pyplot as plt

In [2]:
def load_dataset(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [3]:
def compareModels(models, X, y , metric):
    for i, model in enumerate(models):
        print(f"Model {i}")
        start = time.perf_counter()
        model.fit(X[0], y[0])
        end = time.perf_counter()
        print(f"Time to fit : {end - start} s")
        print(f"Performance : {metric(model.predict(X[1]), y[1])}")


In [4]:
df = load_dataset("citiesSmall.pkl")
X = df['X']
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)
X_train.shape

(300, 2)

In [5]:
depth = [i for i in range(1, 24)]
accuracy = [[], []]
execution_time = [[], []]

for i in depth:
    print("==========================================")
    models = []
    models.append(DecisionTreeClassifier(criterion='gini', max_depth=i))
    models.append(decision_tree.DecisionTreeClassifier(criterion='gini', max_depth=i))

    compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)
    

Model 0
Time to fit : 0.0016870999988896074 s
Performance : 0.67
Model 1
Time to fit : 0.04923620000045048 s
Performance : 0.67
Model 0
Time to fit : 0.0010306000003765803 s
Performance : 0.75
Model 1
Time to fit : 0.0638846999991074 s
Performance : 0.75
Model 0
Time to fit : 0.0007820000009814976 s
Performance : 0.8
Model 1
Time to fit : 0.09770550000030198 s
Performance : 0.79
Model 0
Time to fit : 0.0009594999992259545 s
Performance : 0.81
Model 1
Time to fit : 0.11720319999949425 s
Performance : 0.8
Model 0
Time to fit : 0.0008581000001868233 s
Performance : 0.84
Model 1
Time to fit : 0.13740560000042024 s
Performance : 0.85
Model 0
Time to fit : 0.0008433000002696645 s
Performance : 0.82
Model 1
Time to fit : 0.14634189999924274 s
Performance : 0.82
Model 0
Time to fit : 0.0008766999999352265 s
Performance : 0.86
Model 1
Time to fit : 0.14649070000086795 s
Performance : 0.85
Model 0
Time to fit : 0.0008591000005253591 s
Performance : 0.86
Model 1
Time to fit : 0.16213500000048953 

In [4]:
X, y = datasets.make_classification(n_samples=1000, n_features=5, n_informative=3, n_redundant=2, n_classes=3, random_state=19)

X_categorical = pd.DataFrame()
for i in range(X.shape[1]):
    categorical_feature = pd.cut(X[:, i], bins=5, labels=[j for j in range(5)])
    X_categorical[f'feature_{i}'] = categorical_feature

X = X_categorical.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)

depth = [i for i in range(1, 24)]
accuracy = [[], []]
execution_time = [[], []]

for i in depth:
    print(f"Depth {i} ==========================================")
    models = []
    models.append(RandomForestClassifier(criterion='gini', max_depth=i))
    models.append(random_forest.RandomForestClassifier(criterion='gini', max_depth=i))

    compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)

Depth 1 ==========================================
Model 0
Time to fit : 0.1501496999990195 s
Performance : 0.704
Model 1
Time to fit : 1.6347561000002315 s
Performance : 0.556
Depth 2 ==========================================
Model 0
Time to fit : 0.12973039998905733 s
Performance : 0.804
Model 1
Time to fit : 0.23494349999236874 s
Performance : 0.796
Depth 3 ==========================================
Model 0
Time to fit : 0.14798979999613948 s
Performance : 0.812
Model 1
Time to fit : 0.47432529999059625 s
Performance : 0.784
Depth 4 ==========================================
Model 0
Time to fit : 0.14553339999110904 s
Performance : 0.836
Model 1
Time to fit : 0.47217300000193063 s
Performance : 0.792
Depth 5 ==========================================
Model 0
Time to fit : 0.117544300010195 s
Performance : 0.836
Model 1
Time to fit : 0.5295675999950618 s
Performance : 0.824
Depth 6 ==========================================
Model 0
Time to fit : 0.12342629999329802 s
Performance : 0

In [10]:
X, y = datasets.make_regression(n_samples=10000, n_features=10, noise=1, bias=19, random_state=4)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

models = [LinearRegression(), linear_regression.LinearRegression()]
compareModels(models, (X_train, X_test), (y_train, y_test), mean_squared_error)

# fig = plt.figure(figsize=(10, 8))
# plt.scatter(X, y, marker='o')
# plt.plot(X, model.predict(X), marker = 'o')
# plt.show()

Model 0
Time to fit : 0.05425850000028731 s
Performance : 1.0001200963464363
Model 1
Time to fit : 0.4176121999998941 s
Performance : 1.0001200963464367


In [7]:
X, y = datasets.make_classification(n_samples=18000, n_features=5, n_informative=5, n_redundant=0, n_classes=2, random_state=19)

X_categorical = pd.DataFrame()
for i in range(X.shape[1]):
    categorical_feature = pd.cut(X[:, i], bins=5, labels=[j for j in range(5)])
    X_categorical[f'feature_{i}'] = categorical_feature

X = X_categorical.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)

models = [MultinomialNB(), naive_bayes.NaiveBayes()]
compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)

Model 0
Time to fit : 0.007010999999693013 s
Performance : 0.7906666666666666
Model 1
Time to fit : 0.06470350000017788 s
Performance : 0.8915555555555555


In [4]:
X, y = datasets.make_classification(n_samples=1000, n_features=5, n_informative=3, n_redundant=2, n_classes=3, random_state=19)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)

for k in range(5, 101):
    print(f"\n{k} neighbors ============================\n")
    models = [KNeighborsClassifier(n_neighbors=k), nearest_neighbors.KNN(k)]
    compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)


5 neighbors ============================

Model 0
Time to fit : 0.002048400000148831 s
Performance : 0.876
Model 1
Time to fit : 5.069999997431296e-05 s
Performance : 0.876

6 neighbors ============================

Model 0
Time to fit : 0.0010681000003387453 s
Performance : 0.876
Model 1
Time to fit : 2.8700000257231295e-05 s
Performance : 0.876

7 neighbors ============================

Model 0
Time to fit : 0.0014795000001868175 s
Performance : 0.88
Model 1
Time to fit : 2.9799999992974335e-05 s
Performance : 0.88

8 neighbors ============================

Model 0
Time to fit : 0.001076900000043679 s
Performance : 0.88
Model 1
Time to fit : 2.8700000257231295e-05 s
Performance : 0.88

9 neighbors ============================

Model 0
Time to fit : 0.0011988000001110777 s
Performance : 0.872
Model 1
Time to fit : 3.150000020468724e-05 s
Performance : 0.872

10 neighbors ============================

Model 0
Time to fit : 0.0013005999999222695 s
Performance : 0.876
Model 1
Time to fi

In [3]:
from deep_learning import *
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X.shape)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

mlp = Module()

# Add layers to your MLP model
mlp.add_layer(Layer(input_dim=X_train.shape[1], output_dim=64))  # Adjust the dimensions as needed
mlp.add_layer(Sigmoid(64, 64))  # Add activation layers and adjust dimensions as needed
mlp.add_layer(Layer(64, 1))

# Fit the model to the training data
mlp.fit(X_train, y_train)

# Test the model on the testing data
predictions = mlp.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

(150, 4)
layer : <class 'deep_learning.Layer'>
	input : (32, 4)
	output : (32, 64)
layer : <class 'deep_learning.Sigmoid'>
	input : (32, 64)
	output : (32, 64)
layer : <class 'deep_learning.Layer'>
	input : (32, 64)
	output : (32, 3)


ValueError: operands could not be broadcast together with shapes (32,3) (32,) 